In [142]:
import os
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from astropy.table import Table
from automated_scripts.cal_uvis_make_darks import fileIO
from automated_scripts.cal_uvis_make_darks import dark_stats as ds
from automated_scripts.cal_uvis_make_darks import monitoring_bokeh_plots as mbp

### Making the median dark current .dat files for unflashed and postflashed ref files

In [17]:
# first for unflashed superdarks
uf_loc = '/grp/hst/wfc3e/jmedina/uf_pl/superdarks'# location of the unflashed superdarks
superdarks = glob(os.path.join(uf_loc,'*'))
"""
expstarts = []
midpts_ext1 = []
midpts_ext4 = []

for superdark in superdarks:
    expstart, midpt_ext1, midpt_ext4 = ds.get_dark_median(superdark, 'superdarks')
    expstarts.append(expstart)
    midpts_ext1.append(midpt_ext1)
    midpts_ext4.append(midpt_ext4)
 
# putting into a table
t = Table([expstarts, midpts_ext1, midpts_ext4], names=('expstart', 'midpt_ext1', 'midpt_ext4'))
t.write('uf_midpt.dat', format='ascii')
""" 

In [79]:
from glob import glob
import logging
import os

from astropy.io import fits
import numpy as np

def calc_stats(stat_type, paths, anneal_date, ctecorr):
    """Builds a file within the `blvs` and `superdarks` stats subdirectories
    containing either dark current or hot pixel statistics, based on
    ``stat_type``.
    Creates 2 output statistic files (one under ``blvs`` and one under
    ``superdarks``), removing any already-existing ones, containing either dark
    current statistics (if ``stat_type`` is ``midpt``) or hot pixel statistics
    (if ``stat_type`` is ``hotpix``).
    Parameters
    ----------
    stat_type : str
        Can be either ``midpt`` to calculate dark current statistics
        or ``hotpix`` to calculate hot pixel statistics.
    paths : dict
        The dictionary containing the absolute paths of directories
        used throughout the pipeline.
    anneal_date : str
        The anneal date, in the format of ``YYYYMMDD``, to be used in
        the filename of the output file.
    ctecorr : bool
        ``True`` (if CTE-correction is turned on) or ``False`` (if
        CTE-correction is turned off). Used to determine the output
        filename.
    Notes
    -----
    Statistic files are replaced if they already exist so that the
    files are essentially updated, in case the pipeline is run multiple
    times throughout an anneal cycle.
    """

    # The process is performed twice:
    # 1) Using the blv_tmp.fits files to calculate the statistics
    # 2) Using the superdark files to calculate the statistics
    for data_type in ['superdarks']:

        # Build path to stat files we're about to create
        if ctecorr:
            path_ext1 = os.path.join(paths['plot_'+data_type], '{}/{}_stat_ext1_{}_ctecorr.dat'.format(stat_type, stat_type, anneal_date))
            path_ext4 = os.path.join(paths['plot_'+data_type], '{}/{}_stat_ext4_{}_ctecorr.dat'.format(stat_type, stat_type, anneal_date))
        else:
            path_ext1 = os.path.join(paths['plot_'+data_type], '{}/{}_stat_ext1_{}.dat'.format(stat_type, stat_type, anneal_date))
            path_ext4 = os.path.join(paths['plot_'+data_type], '{}/{}_stat_ext4_{}.dat'.format(stat_type, stat_type, anneal_date))

        # Remove stat files if they already exist
        if os.path.exists(path_ext1):
            os.remove(path_ext1)
        if os.path.exists(path_ext4):
            os.remove(path_ext4)

        # Calculate stats
        if stat_type == 'midpt':
            logging.info('\tCalculating dark median statistics.')
            logging.info('')
            logging.info('\t\tFilename\tEXPSTART\tDark median ext 1\tdark median ext4')
            files = glob(paths[data_type+'_for_dark'])
            for frame in files:
                expstart, stats_ext1, stats_ext4 = ds.get_dark_median(frame, data_type)
                filename = os.path.basename(frame)
                logging.info('\t\t{}\t{}\t{}\t{}'.format(filename, str(expstart), str(stats_ext1), str(stats_ext4)))
                ds.write_to_output(filename, expstart, stats_ext1, path_ext1)
                ds.write_to_output(filename, expstart, stats_ext4, path_ext4)
            logging.info('')
            logging.info('\tDark median statistic files written to {}'.format(os.path.join(paths['plot_'+data_type], 'midpt/')))
            logging.info('')

        elif stat_type == 'hotpix':
            logging.info('\tCalculating hot pixel statistics.')
            logging.info('')
            logging.info('\t\tFilename\tEXPSTART\tPercent hotpix ext 1\tPercent hotpix ext4')
            files = glob(paths[data_type+'_for_hotpix'])
            for frame in files:
                expstart, stats_ext1, stats_ext4 = ds.get_hotpix_stats(frame, data_type)
                filename = os.path.basename(frame)
                logging.info('\t\t{}\t{}\t{}\t{}'.format(filename, str(expstart), str(stats_ext1), str(stats_ext4)))
                ds.write_to_output(filename, expstart, stats_ext1, path_ext1)
                ds.write_to_output(filename, expstart, stats_ext4, path_ext4)
            logging.info('')
            logging.info('\tHot pixel statistic files written to {}'.format(os.path.join(paths['plot_'+data_type], 'hotpix/')))
            logging.info('')

In [158]:
for i in cte_cycles:
    #print(i)
    print(i.split('/')[-1].split('-')[-1].split('_')[0])

20090821
20090914
20091016
20091114
20091211
20100106
20100204
20100302
20100331
20100430
20100526
20100621
20100721
20100820
20100918
20101015
20101114
20101209
20110107
20110205
20110303
20110330
20110428
20110526
20110624
20110721
20110819
20110918
20111015
20111111
20111205
20120104
20120204
20120229
20120328
20120425
20120523
20120620
20120717
20120815
20120911
20121011
20121108
20121206
20130101
20130131
20130228
20130326
20130426
20130522
20130620
20130718
20130815
20130912
20131010
20131110
20131204
20140102
20140130
20140226
20140328
20140423
20140520
20140618
20140715
20140813
20140911
20141010
20141105
20141204
20141231
20150129
20150228
20150323
20150422
20150520
20150620
20150715
20150812
20150908
20151005
20151104
20151202
20151230
20151230
20160125
20160125
20160222
20160222
20160324
20160324
20160419
20160521
20160615
20160713
20160812
20160907
20161008
20161008
20161105
20170223
20170320
20170419
20170518
20170613
20170714
20170810
20170810
20170904
20170904
20171004
2

In [ ]:
# then for postflashed superdarks
# note: we are making the actual .dat files to put under /new_algorithm/superdarks/midpt
# directory for the quicklook darks monitor project

pf_loc = '/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/'# location of the
                                                           # postflash anneal cycles
anneal_cycles = glob(os.path.join(pf_loc,'post*'))
anneal_cycles = np.sort(anneal_cycles)



cycles_to_use = []
cte_cycles = []
for anneal_cycle in anneal_cycles:   
    if 'ctecorr' not in anneal_cycle:
        cycles_to_use.append(anneal_cycle)
    elif 'ctecorr' in anneal_cycle:
        cte_cycles.append(anneal_cycle)


for cycle in cte_cycles:
    anneal_date = cycle.split('/')[-1].split('-')[-1].split('_')[0]
    path = fileIO.make_path_dict(cycle)
    
    calc_stats('midpt', path, anneal_date, True)
    calc_stats('hotpix', path, anneal_date, True) 
"""
    pf_superdarks = glob(path['superdarks_for_dark']) # the superdarks delivered 
                                                      # for 'cycle'

    pf_expstarts = []
    pf_midpts_ext1 = []
    pf_midpts_ext4 = []
    
    for pf_superdark in pf_superdarks:
        pf_expstart, pf_midpt_ext1, pf_midpt_ext4 = ds.get_dark_median(pf_superdark, 'superdarks')
        pf_expstarts.append(expstart)
        pf_midpts_ext1.append(midpt_ext1)
        pf_midpts_ext4.append(midpt_ext4)
        
    t_ext1 = Table([os.path.basename(pf_superdark), 
                    expstart, 
                    midpts_ext4], 
                   names=('filename', 
                          'midpt_ext1', 
                          'midpt_ext1'))
    t.write('midpt_stat', format='ascii')
    """

In [119]:
pf_stats_loc = '/grp/hst/wfc3h/bourque/uvis_darks/plots/new_algorithm/superdarks/midpt'

datfiles = glob(os.path.join(pf_stats_loc, '*.dat'))

dats_to_use_ext1 = []
dats_to_use_ext4 = []

# get all our .dat files in a list
for datfile in datfiles:
    if ('ctecorr' not in datfile) & ('-' not in os.path.basename(datfile)) \
    & ('ext1' in os.path.basename(datfile)):
        dats_to_use_ext1.append(datfile)
    if ('ctecorr' not in datfile) & ('-' not in os.path.basename(datfile)) \
    & ('ext4' in os.path.basename(datfile)):
        dats_to_use_ext4.append(datfile)
  
pf_midpts_ext1 = []
pf_midpts_ext4 = []
pf_expstarts1   = []
pf_expstarts4   = []
# pull out x and y from every .dat file 
for dat1 in dats_to_use_ext1:
    t = Table.read(dat1, format='ascii')
    pf_expstart = t['col2'] #+ 54993 # t['col2'] is actually "days since MJD 54993"
                                     # so we gotta add the 54993 back in, to be consistent
                                     # with my x values for the unflashed darks
    pf_midpt_ext1 = t['col3']
    
    pf_expstarts1.extend(pf_expstart)
    pf_midpts_ext1.extend(pf_midpt_ext1)
    
for dat4 in dats_to_use_ext4:
    t = Table.read(dat4, format='ascii')
    pf_expstart = t['col2']
    pf_midpt_ext4 = t['col3']
    pf_expstarts4.extend(pf_expstart)
    pf_midpts_ext4.extend(pf_midpt_ext4)

"""
expstarts = []
midpts_ext1 = []
midpts_ext4 = []

for superdark in superdarks:
    expstart, midpt_ext1, midpt_ext4 = ds.get_dark_median(superdark, 'superdarks')
    expstarts.append(expstart)
    midpts_ext1.append(midpt_ext1)
    midpts_ext4.append(midpt_ext4)
"""

"\nexpstarts = []\nmidpts_ext1 = []\nmidpts_ext4 = []\n\nfor superdark in superdarks:\n    expstart, midpt_ext1, midpt_ext4 = ds.get_dark_median(superdark, 'superdarks')\n    expstarts.append(expstart)\n    midpts_ext1.append(midpt_ext1)\n    midpts_ext4.append(midpt_ext4)\n"

In [62]:
path = fileIO.make_path_dict(cycles_to_use[-1])
ds.dark_stats_main(cycles_to_use[0]

'/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/post-anneal-20090821_20150728'

In [ ]:
"""
# then for postflashed superdarks
# note: we are making the actual .dat files to put under /new_algorithm/superdarks/midpt
# directory for the quicklook darks monitor project

pf_loc = '/grp/hst/wfc3h/bourque/uvis_darks/new_algorithm/'# location of the
                                                           # postflash superdarks
directories = glob(os.path.join(pf_loc, 'post*'))
paths_to_use = []
for i in directories:
    if ('to' not in i) & ('ctecorr' not in i):
        paths_to_use.append(i)
pf_superdarks = []
for path in paths_to_use:
    superdarks = glob(os.path.join(main_dir,path,'deliver', '*drk*'))
    pf_superdarks.extend(superdarks)

pf_expstarts = []
pf_midpts_ext1 = []
pf_midpts_ext4 = []

for pf_superdark in pf_superdarks:
    expstart, midpt_ext1, midpt_ext4 = ds.get_dark_median(superdark, 'superdarks')
    pf_expstarts.append(expstart)
    pf_midpts_ext1.append(midpt_ext1)
    pf_midpts_ext4.append(midpt_ext4)
    """

# Plotting

In [150]:
anneal_list = mbp.get_history_list('anneal')
anneal_begin = list(range(0, len(anneal_list), 2))
anneal_end = list(range(1, len(anneal_list), 2))

anneal_begin = np.asarray(anneal_begin)
anneal_end = np.asarray(anneal_end)

for i,j in zip(anneal_begin, anneal_end):
    print(i, j)
"""
    plt.axhspan(i, j, facecolor='0.2', alpha=0.5)
    plt.axvspan(i, j, facecolor='b', alpha=0.5)
        """

0 1
2 3
4 5
6 7
8 9
10 11
12 13
14 15
16 17
18 19
20 21
22 23
24 25
26 27
28 29
30 31
32 33
34 35
36 37
38 39
40 41
42 43
44 45
46 47
48 49
50 51
52 53
54 55
56 57
58 59
60 61
62 63
64 65
66 67
68 69
70 71
72 73
74 75
76 77
78 79
80 81
82 83
84 85
86 87
88 89
90 91
92 93
94 95
96 97
98 99
100 101
102 103
104 105
106 107
108 109
110 111
112 113
114 115
116 117
118 119
120 121


"\n    plt.axhspan(i, j, facecolor='0.2', alpha=0.5)\n    plt.axvspan(i, j, facecolor='b', alpha=0.5)\n        "

In [146]:
%matplotlib qt

#import seaborn as sns

#sns.set_style('darkgrid')
# adding sic&dh lockup lines
anneal_list = mbp.get_history_list('anneal')
anneal_begin = list(range(0, len(anneal_list), 2))
anneal_end = list(range(1, len(anneal_list), 2))

ymin = 0 # bottom part of the shade
ymax = 16 # top part of the shade
xmax = 0 # not sure why xmax is a parameter since it's not used but yeah
# adding shaded anneals 
# adding postflash start


# plotting data
fig, axs = plt.subplots(nrows=2,ncols=1,sharex=True, sharey=True, figsize=(6,12))
plt.ylim(ymin,ymax)
# chip 1
axs[0].set_title('Chip 1')
axs[0].set_ylabel(r'Median Dark Current ($\frac{e^-}{hr}$)')
axs[0].set_xlabel('MJD')

axs[0].scatter(expstarts, midpts_ext4, s=20.0,color='coral', alpha=1, edgecolor='black',
            linewidth=0.3)
axs[0].scatter(pf_expstarts4, pf_midpts_ext4, s=20.0,color='grey', alpha=1, edgecolor='black',
               linewidth=0.3)

axs[0].fill_between

# chip 2
axs[1].set_title('Chip 2')
axs[1].set_ylabel(r'Median Dark Current ($\frac{e^-}{hr}$)')
axs[1].set_xlabel('MJD')
axs[1].scatter(expstarts, midpts_ext1, s=20.0,color='coral', alpha=1, edgecolor='black',
            linewidth=0.3)
axs[1].scatter(pf_expstarts1, pf_midpts_ext1, s=20.0,color='grey', alpha=1, edgecolor='black',
            linewidth=0.3)

# IMPORTANT: chip 1 n 2 need different expstart arrays... meaning they're not the 
# same on both chips fo rthe same observation. this doesnt make sense. investigate

11.3637000000017
39.11970000000292
70.40570000000298
95.14570000000094
126.81169999999838
155.5066999999981
183.11669999999867
208.99270000000251
238.1137000000017
264.20270000000164
293.0327000000034
322.9357000000018
349.2436999999991
376.2737000000052
404.56870000000345
434.6307000000015
463.44970000000467
490.8597000000009
520.6926999999996
545.879700000005
574.5717000000004
603.6827000000048
629.9637000000002
656.9607000000033
685.504700000005
713.8187000000034
742.379700000005
769.9936999999991
798.8187000000034
828.5027000000046
855.5517000000036
882.5837000000029
907.2377000000051
937.2687000000005
967.9297000000006
993.0097000000023
1020.7897000000012
1049.2197000000015
1077.2687000000005
1104.5857000000033
1131.7527000000046
1160.9176999999981
1187.4847000000009
1218.020700000001
1245.415700000005
1273.981700000004
1299.9667000000045
1329.4447
1357.8247000000047
1383.8277000000016
1414.6057
1441.1336999999985
1469.6797000000006
1497.464700000004
1525.381699999998
1553.5976999

In [106]:
pf_midpts_ext4

[<Column name='col3' dtype='float64' length=22>
 4.3711
  4.373
 4.3738
 4.3691
 4.3692
 4.3687
  4.372
 4.3713
 4.3727
 4.3708
 4.3722
 4.3728
 4.3724
 4.3691
 4.3722
 4.3705
 4.3737
 4.3735
 4.3704
 4.3715
 4.3743
 4.3743, <Column name='col3' dtype='float64' length=24>
 4.0784
 4.0786
 4.0764
 4.0789
 4.0772
 4.0798
 4.0749
 4.0773
 4.0772
 4.0804
 4.0804
 4.0781
 4.0768
 4.0799
 4.0769
 4.0783
 4.0744
 4.0795
 4.0802
  4.077
 4.0801
   4.08
 4.0788
 4.0756, <Column name='col3' dtype='float64' length=23>
 6.7832
 6.7783
 6.7765
 6.7832
 6.7903
 6.7798
 6.7802
 6.7865
 6.7787
 6.7879
 6.7757
 6.7834
 6.7765
 6.7792
 6.7783
 6.7783
   6.78
 6.7824
 6.7847
 6.7833
  6.778
 6.7779
 6.7763, <Column name='col3' dtype='float64' length=25>
 4.4369
 4.4362
   4.44
  4.439
 4.4376
 4.4387
 4.4396
 4.4402
 4.4368
 4.4391
 4.4404
 4.4362
 4.4385
 4.4389
 4.4393
 4.4393
  4.439
 4.4361
 4.4386
 4.4395
 4.4371
 4.4386
 4.4406
 4.4365
 4.4389, <Column name='col3' dtype='float64' length=21>
 6.1389


In [108]:
expstarts

[2299.295768099997,
 1670.2651394900022,
 2859.2366307600023,
 2411.559579059998,
 2691.598000209997,
 2314.2819146999973,
 1881.1717413799997,
 2999.217357380003,
 1642.487207459999,
 1460.3530824199988,
 2873.319455680001,
 2342.9816491300007,
 2663.4653490499986,
 2482.142555630002,
 2230.2167389399983,
 2733.549231830002,
 2398.312900539997,
 1810.3807214699991,
 2034.26570412,
 2271.4592514800024,
 1684.4290513399974,
 1698.529515000002,
 2495.258007110002,
 1503.450884769998,
 1936.3812569200018,
 1530.3094850100024,
 2971.459668919997,
 2523.103308509999,
 1712.3103368599986,
 2846.2097903899994,
 2075.692440769999,
 2943.0990292700008,
 2747.4159214600004,
 2103.661851509998,
 1474.1273656400008,
 1908.2438487600011,
 3055.121947530002,
 2426.4557834200023,
 1586.770319590003,
 1838.4066944099977,
 2803.132716979999,
 1517.10738958,
 2258.2543670399973,
 2454.588145230002,
 2818.0209704399967,
 2286.3591710600012,
 2383.2328305799965,
 1740.41370569,
 2831.2868500699988,
 2789.

In [112]:
pf_expstarts

[57946.012443,
 57936.060048,
 57921.083219,
 57938.09183,
 57944.056911,
 57944.056911,
 57940.063126,
 57940.063126,
 57922.198612,
 57940.063126,
 57944.056911,
 57936.060048,
 57930.557721,
 57921.083219,
 57924.000522,
 57930.557721,
 57944.056911,
 57927.285754,
 57940.063126,
 57930.557721,
 57921.083219,
 57946.012443,
 57927.285754,
 57930.557721,
 57938.09183,
 57927.285754,
 55252.224838,
 55240.293866,
 55248.481285,
 55246.44698,
 55235.407211,
 55251.274722,
 55250.284804,
 55255.409966,
 55247.60088,
 55239.016435,
 55243.216111,
 55239.501355,
 55242.495787,
 55249.300417,
 55241.219364,
 55233.980128,
 55237.504942,
 55234.440498,
 55244.424514,
 55253.368148,
 55232.58323,
 55236.576077,
 55245.421123,
 56839.529716,
 56846.542401,
 56836.062169,
 56849.157377,
 56837.115074,
 56846.542401,
 56827.483975,
 56838.709137,
 56842.046012,
 56835.047748,
 56834.092794,
 56848.000514,
 56832.008419,
 56845.021347,
 56831.136776,
 56841.032389,
 56843.625664,
 56843.00806,
 

In [115]:
expstarts

[2299.295768099997,
 1670.2651394900022,
 2859.2366307600023,
 2411.559579059998,
 2691.598000209997,
 2314.2819146999973,
 1881.1717413799997,
 2999.217357380003,
 1642.487207459999,
 1460.3530824199988,
 2873.319455680001,
 2342.9816491300007,
 2663.4653490499986,
 2482.142555630002,
 2230.2167389399983,
 2733.549231830002,
 2398.312900539997,
 1810.3807214699991,
 2034.26570412,
 2271.4592514800024,
 1684.4290513399974,
 1698.529515000002,
 2495.258007110002,
 1503.450884769998,
 1936.3812569200018,
 1530.3094850100024,
 2971.459668919997,
 2523.103308509999,
 1712.3103368599986,
 2846.2097903899994,
 2075.692440769999,
 2943.0990292700008,
 2747.4159214600004,
 2103.661851509998,
 1474.1273656400008,
 1908.2438487600011,
 3055.121947530002,
 2426.4557834200023,
 1586.770319590003,
 1838.4066944099977,
 2803.132716979999,
 1517.10738958,
 2258.2543670399973,
 2454.588145230002,
 2818.0209704399967,
 2286.3591710600012,
 2383.2328305799965,
 1740.41370569,
 2831.2868500699988,
 2789.